# Word Embeddings

## ¿Cómo representamos palabras, oraciones y significados en NLP?

## ¿Qué es una palabra?


![](img/words.png)

Cuando hablamos de palabras, podemos distinguir dos conceptos diferentes:

- **ocurrencia** (*token*) se refiere a una observación de una palabra en una cadena de texto. 

    Como hemos visto, en algunas lenguas es más o menos complejo identificar los límites de las palabras, pero en la mayoría de las lenguas occidentales y de nuestro entorno se utilizan espacios y otros signos de puntuación para delimitar las palabras.

- **tipo** (*type*) es la representación abstracta de una palabra. Cad **ocurrencia** pertenece a un **tipo** de palabra. Cuando contamos la frecuencia de las palabras de un *corpus* o colección de textos, lo que hacemos es contar el número de ocurrencias que tiene cada tipo.

In [ ]:
from nltk import word_tokenize

texts = [
    """No hubo sorpresa en Bruselas. 621 votos a favor, 49 en contra (los 'remainers' británicos entre ellos) y 13 abstenciones.""",
    """'The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced "no evidence" that any irregularities took place.'""",
    """環太平洋造山帯に属する小スンダ列島の西端に位置している。""",
]

for text in texts:
    print(word_tokenize(text))

In [ ]:
tweets = [
    """🎉¡#SORTEO! Gana una tostadora YummyToast Double. 🎁 
▪️Síguenos. 
▪️Comenta mencionando a 2 amigos junto a #Cecotec.
Tienes hasta el 9 de febrero para participar. El regalo se sorteará aleatoriamente entre los participantes. ¡Mucha suerte!.""",
    """we play for y’all 🏀‼️🖤 https://t.co/sd12vW93 #MambaMentality""",
]

for tweet in tweets:
    print(word_tokenize(tweet))

In [ ]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()

for text in texts:
    print(tokenizer.tokenize(text))

for tweet in tweets:
    print(tokenizer.tokenize(tweet))

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

for text in texts + tweets:
    print(tokenize(text))

Veamos qué tipo de tokenización se prefiere cuando son humanos los que segmentan las palabras: el [corpus de Brown](https://en.wikipedia.org/wiki/Brown_Corpus) en inglés, o [Ancora](http://clic.ub.edu/corpus/es) en español.

In [ ]:
from nltk.corpus import brown

brown_sents = brown.tagged_sents(categories="news")
for sentence in brown_sents[:3]:
    print([token for token, _tag in sentence])

In [ ]:
from nltk.corpus import cess_esp

ancora_sents = cess_esp.tagged_sents()
for sentence in ancora_sents[:3]:
    print([token for token, _tag in sentence])

## Representaciones discretas

A partir de aquí vamos a asumir que tenemos solucionado el proceso de tokenización e identificación de lo que es una palabra. ¿Cómo continuamos?

La manera más sencilla de representar una palabra es una cadena, es decir, como una secuencia ordenada de caracteres. Esto es cómodo, pero implica dos cosas:

- La cantidad de memoria que ocupa cada cada palabra varía en función de la longitud :-/

- Comprobar si dos palabras son idénticas es un proceso lento :-(

Otra opción alternativa consiste en representar las palabras como números enteros, de manera que a cada palabra se le asigna de manera más o menos arbitraria un número entero positivo.

![](img/words-indexes.jpg)

In [ ]:
import numpy as np

from sklearn.datasets import fetch_20newsgroups

categories = ["comp.windows.x", "rec.sport.baseball", "sci.space", "talk.religion.misc"]
remove = ("headers", "footers", "quotes")

newsgroups_train = fetch_20newsgroups(
    subset="train", categories=categories, remove=remove
)

newsgroups_train.filenames.shape

In [ ]:
for doc in newsgroups_train.data[:3]:
    print(doc[:300])
    print("-" * 100)

for doc in newsgroups_train.data[-3:]:
    print(doc[:300])
    print("-" * 100)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

tweets_tokens = []
tweets_tokens.extend([tokenize(tweet) for tweet in tweets][0])
print(tweets_tokens)

In [ ]:
label_encoder = LabelEncoder()

tokens_int = label_encoder.fit_transform(tweets_tokens)

token2int = dict(zip(tweets_tokens, tokens_int))
print(token2int)

Este tipo de representación se caracteriza porque:

- Todos las palabras ocupan la cantidad de memoria.

- Comprobar si dos cadenas son la misma palabra es rápido :-)

- Estos identificadores arbitrarios no significan nada :-(

- No hay manera de relacionar palabras similares atendiendo a su identificador :-(

In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)

# aplanamos los tokens_int
tokens_int = tokens_int.reshape(len(tokens_int), 1)
onehot_tokens = onehot_encoder.fit_transform(tokens_int)

print(onehot_tokens)

print(onehot_tokens[token2int["suerte"]])

In [ ]:
from keras.utils import to_categorical

onehot_tokens = to_categorical(tokens_int)

print(onehot_tokens)
print(onehot_tokens[token2int["suerte"]])

Vectorizar colecciones de documentos es una práctica habitual. Veamos un ejemplo:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words="english")

vectors = vectorizer.fit_transform(
    newsgroups_train.data
).todense()  # (documents, vocab)

vectors.shape

Hemos convertido la collección de documentos en una matriz de datos donde los documentos se representan como vectores de enteros.

![](img/vectorized-docs.png)

In [ ]:
vectors[0]

In [ ]:
vocab = np.array(vectorizer.get_feature_names())
vocab.shape

In [ ]:
print(vocab[:50])

In [ ]:
print(vocab[20000:20050])

In [ ]:
print(vocab[-50:])

## Palabras como vectores

La verdad es que hoy en día nunca se representamos las palabras como elementos discretos principalmente por un motivo: muchas veces no necesitaremos comprobar no si dos palabras son iguales, como mencionábamos antes, pero sí si dos palabras son *similares*. Calcular la **similitud** entre dos pares de palabras/oraciones/documentos es crucial para muchas tareas de NLP.

### ¿Qué significa **similitud**?

La idea de similitud implica que dos palabras tienen algún tipo de relación desde el punto de vista del significado, no necesariamente de sinonimia. Que dos palabras sean similares no implica que sean sinónimas e intercambiables en cualquier situación. 

- Dos adjetivos aparentemente contrarios (p. ej. *blanco* y *negro*) son similares porque se pueden aplicar a los mismos objetos. 
- Un término general y otro más específico (p. ej. *perro* y *caniche*) son similares también. 
- Una parte constitutiva y el todo (p. ej. *dedo* y *mano*) son similares.

Históricamente, ha habido distintos intentos de codificar de manera explícita estas relaciones de similitud a mano. Desde el punto de vista Semántica, el ejemplo más famoso es [Wordnet](https://wordnet.princeton.edu/), una base de datos léxica que almacena palabras, sus significados y las relaciones semánticas que se establecen entre ellas de manera jerárquica. Otras partes de la lingüísitica, como la Sintaxis, estudia la estructura del lenguaje y agrupa las palabras similares bajo clases de palabras o categorías como *nombre*, *verbo*, *adjetivo*, etc. 

Más rencientemente, se ha desarrollado una tendencia (hasta convertirse en dominante) consistente en extraer este tipo de relaciones de manera automática, a través del procesamiento de ingentes colecciones de textos y el análisis de cada palabra en su contexto de aparición.

A partir de cualquiera de estas dos vertientes, o más bien, combinando ambas, podemos llegar a la idea de representar una palabra como un vector. Y podemos elegir la dimensionalidad que mejor se ajuste a nuestros intereses:


- • Each word type may be given its own dimension, and assigned 1 in that dimension (while all other
words get 0 in that dimension). Using dimensions only in this way, and no other, is essentially
equivalent to integerizing the words; it is known as a “one hot” representation, because each word
type’s vector has a single 1 (“hot”) and is otherwise 0.
• For a collection of word types that belong to a known class (e.g., days of the week), we can use a
dimension that is given binary values. Word types that are members of the class get assigned 1 in this
dimension, and other words get 0.
• For word types that are variants of the same underlying root, we can similarly use a dimension to
place them in a class. For example, in this dimension, know, known, knew, and knows would all get
assigned 1, and words that are not forms of know get 0.
• More loosely, we can use surface attributes to “tie together” word types that look similar; examples
include capitalization patterns, lengths, and the presence of a digit.
• If word types’ meanings can be mapped to magnitudes, we might allocate dimensions to try to capture
these. For example, in a dimension we choose to associate with “typical weight” elephant might get
12,000 while cat might get 9. Of course, it’s not entirely clear what value to give purple or throw in
this dimension.




Cuando tenemos un mapeo como este entre palabras y entereos podemos representar cada palabra como un vector de $n$ dimensiones, donde $n$ es el tamaño de vocabulario que manejamos. Estos vectores contendrán muchos $0$ y un único $1$, en la posición que coincida con el índice de la palabra en el vocabulario.

![](img/one-hot-vectors.png)

## Palabras como vectores distribucionales

> “You shall know a word by the company it keeps.”
> — John R. Firth (1957)
>
>“The meaning of a word is its use in the language (…) One cannot guess how a word functions. One has to look at its use, and learn from that.”
>— Ludwig Wittgenstein (1953)


La idea de que podemos analizar el uso de las palabras para deducir sus significado es una idea fundamental en semántica distribucional: la hipótesis distribucional. 

Esta idea inspira muchos algoritmos para aprender repesentaciones numéricas de las palanbras --> *word embeddings*.

## Palabras como vectores
